In [ ]:
import yaml
import math
import googlemaps
import pandas as pd
import matplotlib.pyplot as plt
from scraping.pgSQL_handler import pgSQL
from scraping.Parser import *

pd.set_option('display.max_colwidth', -1)

In [46]:
with open('scraping/configs/hamburg.yaml') as f:
    config = yaml.load(f)

gesucht = Gesucht(config)
SQL = pgSQL(config['postgreSQL'])

In [ ]:
bundesland = 'HH'
destinations = {'hbf': (53.553738, 10.005487), 
                'sternschanze': (53.564001, 9.965852),
                'st. pauli': (53.551025, 9.970045)}

In [ ]:
cursor = SQL.pg_conn.cursor()
cursor.execute("select * from hamburg order by timestamp desc limit 50")
                #where lower(address) like '%eppendorfer%weg%'")
df = pd.DataFrame(cursor.fetchall(), columns=[column.name for column in cursor.description])
df.head()

In [ ]:
apartments = df.sample(n=5)
origins = [(row.lat, row.lng) for i, row in apartments.iterrows()]

In [ ]:
gesucht.gmaps.directions

In [ ]:
import holidays

from datetime import time
from datetime import date
from datetime import datetime
from datetime import timedelta

import numpy as np 

In [ ]:
hh_holidays = holidays.Germany(prov=bundesland)

week = [date.today() + timedelta(days=n) for n in range(7)]

weekend = np.array([day.weekday() >= 5 for day in week])      # make sure all postings have a fair comparison.
holiday = np.array([day in hh_holidays for day in week])      # holidays be cray cray

offset = np.argmax((weekend | holiday) == False)


travel_day = date.today() + timedelta(days=int(offset))
travel_time = datetime.combine(travel_day, time(10, 30))       # again, fair comparison. Weekday at 10 AM

res = gesucht.gmaps.distance_matrix(origins, destinations.values(), departure_time=travel_time, mode='bicycle')

In [ ]:
res

In [ ]:
travel_info = []
for apt, row in zip(apartments.iterrows(), res['rows']):
    row = [{'distance':elem['distance']['value'], 'time':elem['duration']['value']/60} for elem in row['elements']]
    travel_info.append({dest: travel_info for dest, travel_info in zip(destinations.keys(), row)})
travel_info

In [ ]:
apartments